In [16]:
# connecting colab with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# importng libraries
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

**Data processing**

In [19]:
%cd drive/MyDrive/Next\ Word\ Prediction\ Model/
data = pd.read_csv('dataset.csv')

[Errno 2] No such file or directory: 'drive/MyDrive/Next Word Prediction Model/'
/content/drive/MyDrive/Next Word Prediction Model


In [21]:
data.head(3)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30


In [22]:
data.tail(3)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
6505,6506,https://medium.com/better-marketing/content-an...,Content and Marketing Beyond Mass Consumption,How to acquire customers without wasting money...,6506.jpg,207,1,8,Better Marketing,2019-12-05
6506,6507,https://medium.com/better-marketing/5-question...,5 Questions All Copywriters Should Ask Clients...,Save time and effort by…,6507.jpg,253,2,5,Better Marketing,2019-12-05
6507,6508,https://medium.com/better-marketing/how-to-wri...,How To Write a Good Business Blog Post,An A-to-Z guide for non-writers,6508.jpg,147,0,9,Better Marketing,2019-12-05


In [23]:
data['title']

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

In [24]:
tokenizer=Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(data['title'])
words=len(tokenizer.word_index)+1

In [25]:
input_sequences=[]
for line in data['title']:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [26]:
print(input_sequences)

[[5, 566], [5, 566, 61], [5, 566, 61, 2], [5, 566, 61, 2, 435], [5, 566, 61, 2, 435, 1310], [5, 566, 61, 2, 435, 1310, 15], [5, 566, 61, 2, 435, 1310, 15, 3508], [5, 566, 61, 2, 435, 1310, 15, 3508, 3509], [3510, 22], [3510, 22, 783], [3510, 22, 783, 112], [3510, 22, 783, 112, 158], [3510, 22, 783, 112, 158, 15], [3510, 22, 783, 112, 158, 15, 478], [3510, 22, 783, 112, 158, 15, 478, 478], [3510, 22, 783, 112, 158, 15, 478, 478, 1651], [6, 2], [6, 2, 63], [6, 2, 63, 3511], [6, 2, 63, 3511, 193], [3512, 6], [3512, 6, 2], [3512, 6, 2, 232], [3512, 6, 2, 232, 1074], [3512, 6, 2, 232, 1074, 11], [3512, 6, 2, 232, 1074, 11, 2217], [3512, 6, 2, 232, 1074, 11, 2217, 22], [3512, 6, 2, 232, 1074, 11, 2217, 22, 10], [3512, 6, 2, 232, 1074, 11, 2217, 22, 10, 3513], [5, 170], [5, 170, 64], [5, 170, 64, 170], [5, 170, 64, 170, 399], [5, 170, 64, 170, 399, 7], [5, 170, 64, 170, 399, 7, 3514], [5, 170, 64, 170, 399, 7, 3514, 2218], [5, 170, 64, 170, 399, 7, 3514, 2218, 8], [5, 170, 64, 170, 399, 7, 35

In [27]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [28]:
input_sequences[2]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   5, 566,  61,   2],
      dtype=int32)

In [29]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=words)

**Building Model**

In [ ]:
model=Sequential()
model.add(Embedding(words,100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(words,activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
history=model.fit(xs,ys,epochs=2,verbose=1)
print(model)

Epoch 1/2
1358/1358 [==============================] - 289s 209ms/step - loss: 7.4711 - accuracy: 0.0858
Epoch 2/2
1262/1358 [==========================>...] - ETA: 19s - loss: 6.6290 - accuracy: 0.1327